# 🎯 Modelo Demo - Flujo Completo Cristian-Style

**Objetivo**: Replicar el flujo real de trabajo de Cristian (DS)

**Modelo**: Autoencoder para detección de anomalías

**Flujo**: Splunk → EDA → Preprocesamiento → Modelo → Entrenar → Inferir → Telemetría


In [ ]:
# ═══════════════════════════════════════════════════════════════
# IMPORTS - Librerías necesarias
# ═══════════════════════════════════════════════════════════════

import sys
sys.path.append("/srv/notebooks_custom/helpers")

# Helpers custom empresariales
from telemetry_helper import log_metrics, log_training_step, log_error, log_prediction
from metrics_calculator import calculate_all_metrics
from preprocessor import standard_preprocessing

# Librerías estándar ML/DL
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

print("✅ Todos los imports exitosos")


In [ ]:
# ═══════════════════════════════════════════════════════════════
# FUNCIÓN 1: INIT - Inicialización del modelo
# ═══════════════════════════════════════════════════════════════

def init(param):
    """
    Inicializar modelo y parámetros globales
    
    Args:
        param: Diccionario con parámetros del modelo
    
    Returns:
        model: Modelo inicializado
    """
    global model, scaler, n_features
    
    try:
        print(f"🔧 Inicializando modelo con parámetros: {param}")
        n_features = None
        model = None
        scaler = None
        print("✅ Modelo inicializado (autoencoder para detección de anomalías)")
        return model
        
    except Exception as e:
        log_error(model_name='demo_modelo_completo', error_message=str(e), error_type='init')
        raise e


In [ ]:
# ═══════════════════════════════════════════════════════════════
# FUNCIÓN 2: FIT - Entrenamiento del modelo
# ═══════════════════════════════════════════════════════════════

def fit(df, param):
    """
    Entrenar modelo con datos
    
    Args:
        df: DataFrame con datos de entrenamiento
        param: Diccionario con parámetros de entrenamiento
    
    Returns:
        model: Modelo entrenado
    """
    global model, scaler, n_features
    
    try:
        print(f"📊 Datos recibidos: {df.shape}")
        
        # Detectar features
        feature_cols = df.columns.tolist()
        n_features = len(feature_cols)
        X = df[feature_cols].values
        
        # Preprocesamiento
        print("🔧 Preprocesando datos...")
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Split train/val
        X_train, X_val = train_test_split(X_scaled, test_size=0.2, random_state=42)
        
        # Construir modelo (autoencoder)
        print(f"🔧 Construyendo autoencoder con {n_features} features...")
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(32, activation='relu', input_shape=(n_features,)),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(8, activation='relu'),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(n_features, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=param.get('learning_rate', 0.001)),
            loss='mse',
            metrics=['mae', 'mse']
        )
        
        # Entrenar
        epochs = param.get('epochs', 50)
        batch_size = param.get('batch_size', 32)
        
        history = model.fit(
            X_train, X_train,
            validation_data=(X_val, X_val),
            epochs=epochs,
            batch_size=batch_size,
            verbose=0,
            callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
        )
        
        # Calcular métricas
        val_pred = model.predict(X_val, verbose=0)
        mse = np.mean((X_val - val_pred) ** 2)
        mae = np.mean(np.abs(X_val - val_pred))
        
        print(f"📊 MSE: {mse:.4f}, MAE: {mae:.4f}")
        
        # Telemetría
        log_metrics(model_name='demo_modelo_completo', mae=mae, mse=mse)
        log_training_step(model_name='demo_modelo_completo', epoch=epochs, loss=mse)
        
        print("✅ Modelo entrenado exitosamente")
        return model
        
    except Exception as e:
        log_error(model_name='demo_modelo_completo', error_message=str(e), error_type='fit')
        raise e


In [ ]:
# ═══════════════════════════════════════════════════════════════
# FUNCIÓN 3: APPLY - Aplicar modelo a nuevos datos
# ═══════════════════════════════════════════════════════════════

def apply(df):
    """
    Aplicar modelo entrenado para predicción
    
    Args:
        df: DataFrame con datos para predecir
    
    Returns:
        errors: Array con errores de reconstrucción
    """
    global model, scaler
    
    try:
        if model is None or scaler is None:
            raise ValueError("Modelo no entrenado. Ejecuta fit() primero")
        
        feature_cols = df.columns.tolist()
        X = df[feature_cols].values
        X_scaled = scaler.transform(X)
        
        reconstructed = model.predict(X_scaled, verbose=0)
        errors = np.mean((X_scaled - reconstructed) ** 2, axis=1)
        
        log_prediction(model_name='demo_modelo_completo', num_predictions=len(errors))
        
        return errors
        
    except Exception as e:
        log_error(model_name='demo_modelo_completo', error_message=str(e), error_type='apply')
        raise e


In [ ]:
# ═══════════════════════════════════════════════════════════════
# FUNCIÓN 4: SUMMARY - Resumen del modelo
# ═══════════════════════════════════════════════════════════════

def summary(df):
    """
    Generar resumen de métricas del modelo
    
    Args:
        df: DataFrame (opcional)
    
    Returns:
        summary_dict: Diccionario con resumen del modelo
    """
    global model, scaler
    
    try:
        if model is None:
            return {"status": "Model not initialized"}
        
        return {
            "model_type": "Autoencoder",
            "use_case": "Anomaly Detection",
            "trainable_parameters": model.count_params(),
            "layers": len(model.layers),
            "features": n_features if 'n_features' in globals() else None
        }
        
    except Exception as e:
        log_error(model_name='demo_modelo_completo', error_message=str(e), error_type='summary')
        return {"error": str(e)}


# ═══════════════════════════════════════════════════════════════
# FASE 1: EXPLORACIÓN DE DATOS DESDE SPLUNK
# ═══════════════════════════════════════════════════════════════

## Cómo Cristian trabajaría:
1. Consultar datos de Splunk (modo exploratorio)
2. Exploración rápida de los datos (EDA)
3. Seleccionar features relevantes
4. Definir preprocesamiento
5. Crear y entrenar modelo
6. Telemetría automática


In [ ]:
# Consultar datos de Splunk (modo exploratorio/interactivo)
try:
    from dsdlsupport import SplunkSearch
    
    print("🔍 Consultando datos de Splunk...")
    # Cristian consultaría sus datos reales
    search = SplunkSearch.SplunkSearch(
        search='index=demo_anomalias_data | head 500 | table feature_*'
    )
    
    # Obtener como DataFrame
    df = search.as_df()
    
    print(f"✅ Datos cargados: {df.shape}")
    print(f"   Columnas: {df.columns.tolist()}")
    
except Exception as e:
    print(f"⚠️  SplunkSearch no disponible: {e}")
    print("   Generando datos dummy para continuar...")
    
    # Fallback: datos dummy (como backup)
    np.random.seed(42)
    normal_data = np.random.normal(0, 1, (500, 5))
    anomaly_data = np.random.normal(5, 2, (50, 5))
    df = pd.DataFrame(
        np.vstack([normal_data, anomaly_data]), 
        columns=[f'feature_{i}' for i in range(5)]
    )
    print(f"✅ Datos dummy generados: {df.shape}")


In [ ]:
# Exploración rápida de los datos
print("📊 Información del dataset:")
print(df.info())

print("\n📈 Estadísticas descriptivas:")
print(df.describe())

print("\n🔍 Primeras filas:")
print(df.head())


In [ ]:
# Visualización rápida (EDA como Cristian)
plt.figure(figsize=(12, 8))
df.plot(subplots=True, figsize=(10, 6*len(df.columns)), sharex=True)
plt.tight_layout()
plt.show()

print("\n✅ Exploración completada")


# ═══════════════════════════════════════════════════════════════
# NOTA IMPORTANTE
# ═══════════════════════════════════════════════════════════════

**Las funciones `init`, `fit`, `apply`, `summary` están arriba**

**Cuando Cristian ejecuta:**
- `| fit MLTKContainer algo=demo_modelo_completo epochs=20 from feature_* into app:demo_v1`
- DSDL automáticamente llama a `fit(df, param)` con los datos de Splunk
  
**NO necesitas consultar Splunk manualmente en fit/apply**
